# Создание индексированной базы знаний для юридического чат-бота

Этот ноутбук создает FAISS индекс для юридической базы данных и сохраняет его для использования в других проектах.

In [ ]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

hf_token = user_secrets.get_secret("HUGGINGFACE_TOKEN")
login(token = hf_token)

In [ ]:
import json
import torch
import os
import pickle
import faiss
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm
import numpy as np

## Класс для создания эмбеддингов текста

In [ ]:
# Улучшенная функция эмбеддинга с использованием моделей E5 (лучше для многоязычного поиска)
class E5Embedder:
    def __init__(self, model_name="intfloat/multilingual-e5-small", device=None):
        if device is None:
            self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        else:
            self.device = device
        
        print(f"Loading embedding model {model_name} on {self.device}...")
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name).to(self.device)
        self.model.eval()
        print("Embedding model loaded")
    
    def _average_pool(self, last_hidden_states, attention_mask):
        # Take attention mask into account for averaging
        last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
        return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]
    
    def encode(self, texts, batch_size=8, show_progress_bar=True):
        # Prepare storage for embeddings
        all_embeddings = []
        
        # Process in batches to avoid OOM
        for i in tqdm(range(0, len(texts), batch_size), disable=not show_progress_bar):
            batch_texts = texts[i:i+batch_size]
            
            # For E5 models, add prefix for better retrieval performance
            processed_texts = [f"passage: {text}" for text in batch_texts]
            
            # Tokenize
            inputs = self.tokenizer(
                processed_texts,
                padding=True,
                truncation=True,
                max_length=512,
                return_tensors="pt"
            ).to(self.device)
            
            # Get embeddings
            with torch.no_grad():
                outputs = self.model(**inputs)
                embeddings = self._average_pool(outputs.last_hidden_state, inputs["attention_mask"])
                
                # Normalize embeddings
                embeddings = torch.nn.functional.normalize(embeddings, p=2, dim=1)
                all_embeddings.append(embeddings.cpu().numpy())
        
        # Concatenate all embeddings
        return np.vstack(all_embeddings)
    
    def encode_queries(self, queries, batch_size=8):
        # Similar to encode but with "query: " prefix instead of "passage: "
        if isinstance(queries, str):
            queries = [queries]
            
        processed_queries = [f"query: {query}" for query in queries]
        
        inputs = self.tokenizer(
            processed_queries,
            padding=True,
            truncation=True,
            max_length=512,
            return_tensors="pt"
        ).to(self.device)
        
        with torch.no_grad():
            outputs = self.model(**inputs)
            embeddings = self._average_pool(outputs.last_hidden_state, inputs["attention_mask"])
            embeddings = torch.nn.functional.normalize(embeddings, p=2, dim=1)
        
        return embeddings.cpu().numpy()

## Функции для обработки и индексирования документов

In [ ]:
# Загрузка данных
def load_data(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    return data

In [ ]:
# Улучшенная функция разбиения документов на чанки для лучшего поиска
def chunk_documents(legal_data, max_chunk_size=256, overlap=50):
    chunks = []
    references = []
    
    for item in legal_data:
        text = item["Текст"]
        reference = item["Ссылка"]
        
        # Для очень коротких текстов, оставляем как есть
        if len(text.split()) <= max_chunk_size:
            chunks.append(text)
            references.append(reference)
            continue
        
        # Разбиваем более длинные тексты на чанки с перекрытием
        words = text.split()
        current_position = 0
        
        while current_position < len(words):
            end_position = min(current_position + max_chunk_size, len(words))
            chunk = " ".join(words[current_position:end_position])
            
            # Добавляем дополнительную информацию об источнике к каждому чанку
            # Это помогает сохранить контекст даже в чанках
            ref_info = f"{reference} - Фрагмент {current_position//max_chunk_size + 1}"
            
            chunks.append(chunk)
            references.append(ref_info)
            
            # Перемещаемся с перекрытием
            current_position += max_chunk_size - overlap
    
    return chunks, references

In [ ]:
# Создание улучшенного FAISS индекса
def create_improved_faiss_index(legal_data, embedder, chunk_size=256, overlap=50):
    print("Chunking documents...")
    chunks, references = chunk_documents(legal_data, max_chunk_size=chunk_size, overlap=overlap)
    print(f"Created {len(chunks)} chunks from {len(legal_data)} documents")
    
    # Создание эмбеддингов
    print("Creating embeddings...")
    embeddings = embedder.encode(chunks)
    
    # Создание FAISS индекса
    print("Building FAISS index...")
    dimension = embeddings.shape[1]
    
    # Используем Flat индекс для лучшей точности
    index = faiss.IndexFlatIP(dimension)
    index.add(embeddings)
    
    return index, chunks, references

In [ ]:
# Функция для сохранения индекса и компонентов
def save_index_components(index, chunks, references, embedder_model_name, output_path):
    # Создаем директорию, если ее не существует
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    
    # Сохраняем FAISS индекс
    index_path = os.path.join(output_path, "legal_index.faiss")
    faiss.write_index(index, index_path)
    
    # Сохраняем чанки и references
    data_path = os.path.join(output_path, "chunks_references.pkl")
    with open(data_path, "wb") as f:
        pickle.dump({"chunks": chunks, "references": references, "embedder_model": embedder_model_name}, f)
    
    print(f"Индекс и компоненты успешно сохранены в {output_path}")
    print(f"Путь к индексу: {index_path}")
    print(f"Путь к данным: {data_path}")

## Создание и сохранение индекса

In [ ]:
# Выполнение индексации в Kaggle
legal_data_path = "C:/Users/ten-t/Desktop/LegalGuardian/data/json/legal_documents.json"  # Путь к JSON файлу в Kaggle
output_dir = "C:/Users/ten-t/Desktop/LegalGuardian/data/index"  # Место для сохранения индекса в Kaggle
embedding_model_name = "intfloat/multilingual-e5-small"  # Используем small модель

# Инициализируем embedder
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Используем устройство: {device}")
embedder = E5Embedder(model_name=embedding_model_name, device=device)

# Загружаем данные
print(f"Загружаем юридические данные из {legal_data_path}...")
legal_data = load_data(legal_data_path)
print(f"Загружено {len(legal_data)} юридических документов")

In [ ]:
# Создаем индекс
index, chunks, references = create_improved_faiss_index(legal_data, embedder)

# Сохраняем компоненты
save_index_components(index, chunks, references, embedding_model_name, output_dir)

print("Процесс создания и сохранения индекса завершен.")

## Проверка сохраненного индекса

In [ ]:
# Проверяем, что файлы созданы
index_file = os.path.join(output_dir, "legal_index.faiss")
data_file = os.path.join(output_dir, "chunks_references.pkl")

print(f"Проверка файла индекса: {os.path.exists(index_file)}")
print(f"Проверка файла данных: {os.path.exists(data_file)}")

# Выводим размер созданных файлов
if os.path.exists(index_file) and os.path.exists(data_file):
    index_size = os.path.getsize(index_file) / (1024 * 1024)  # размер в МБ
    data_size = os.path.getsize(data_file) / (1024 * 1024)  # размер в МБ
    
    print(f"Размер файла индекса: {index_size:.2f} МБ")
    print(f"Размер файла данных: {data_size:.2f} МБ")
    print(f"Общий размер: {index_size + data_size:.2f} МБ")
    
    # Быстрая проверка данных
    with open(data_file, "rb") as f:
        data = pickle.load(f)
        print(f"\nКоличество чанков: {len(data["chunks"])}")
        print(f"Модель эмбеддингов: {data["embedder_model"]}")
        
    print("\nИндекс успешно создан и готов к использованию в других проектах.")